In [59]:
from openai import OpenAI


In [60]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.environ["OPENAI_API"]

### import DB

import csv into db (https://www.sqlitetutorial.net/sqlite-import-csv/)


In [61]:
from peewee import *

database = SqliteDatabase('postings')

class UnknownField(object):
    def __init__(self, *_, **__): pass

class BaseModel(Model):
    class Meta:
        database = database

class Postings(BaseModel):
    date = TextField(null=True)
    reactions = TextField(null=True)
    text = TextField(null=True)

    class Meta:
        table_name = 'postings'
        primary_key = False


In [62]:
text = []

for posting in Postings.select():
    text.append(posting.text)

In [63]:
reactions = []

for posting in Postings.select():
    reactions.append(posting.reactions)

### Context Extraction

In [64]:
context_prompt_template = """
Extract a writer's personalities, life styles, hobbies from social media posts. The personalities and the hobbies may be multiple attributes.

INPUT: {text}
Output should be a JSON object like this:

{
personalities: [xx, xx, ..],
life styles: [xx, xx, ..],
hobbies: [xx, ...]
}

"""

In [65]:
client = OpenAI( api_key=api_key )

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  response_format=  { "type": "json_object" },
  messages=[
     {
        "role": "system",
        "content": "I am helpful system which is good at extracting information from data"
      },
      {
        "role": "user",
        "content":  context_prompt_template
      }
  ]
)

style = []
style.append(completion.choices[0].message.content)

In [66]:
print(style)

['{\n    "personalities": ["creative", "intellectual", "outgoing"],\n    "life styles": ["entrepreneurial", "adventurous"],\n    "hobbies": ["writing", "hiking", "reading"]\n}']


### Style Extraction

In [67]:
style_prompt_template = """
Extract social media post's language tones, well-used emojis, and general writing topics written from social media posts. The styles and the topics may be multiple attributes.

INPUT: {text}
Output should be a JSON object like this:

{
language tones: [xx, xx, ..],
emojis: [xx, ...],
writing_topics: [xx, ...],
}

"""

In [68]:
client = OpenAI( api_key=api_key )

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  response_format=  { "type": "json_object" },
  messages=[
     {
        "role": "system",
        "content": "I am helpful system which is good at extracting information from data"
      },
      {
        "role": "user",
        "content":  style_prompt_template
      }
  ]
)

style_2 = []
style_2.append(completion.choices[0].message.content)

In [69]:
print(style_2)

['{\n   "language tones": ["informal", "positive", "enthusiastic"],\n   "emojis": ["😂", "🎉", "❤️"],\n   "writing_topics": ["food", "travel", "music"]\n}']


In [70]:
test_comment = "I mean sure some are big or whatever, but honestly, everything looks good on you!! 🫶🏻"

In [77]:
response_prompt_template = """
Write a response to test_comment based on writing style and life style. The response should consider a language tone, emojis, and topic of the {test_comment}, then choose how to response it in style.

life style: {style}
writing_style: {style_2}
test_comment: {test_comment}

""
Output should be a JSON object like this:

{
response: xx
}

"""

In [72]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  response_format=  { "type": "json_object" },
  messages=[
     {
        "role": "system",
        "content": "I am a message writing assistant that write responses considering language tones, emojis, and writing topics"
      },
      {
        "role": "user",
        "content": response_prompt_template
      }
  ]
)

In [73]:
completion.choices[0].message.content

'{\n"response": "Wow, your writing style is so eloquent and sophisticated! 🌟 It\'s clear that you have a deep appreciation for language. Keep shining bright with your unique voice! ✨"\n}'

### Identify intent

In [74]:
intent_prompt_template = """
Extract social media post's writing styles and general writing topics written from social media post. The styles and the topics may be multiple attributes.

INPUT: {reactions}
Output should be a JSON object like this:

{
response: xx
}

"""

### Message suggestion

In [75]:
prompt_template = """

Provide an appropriate response based on given intent of the sender, style of the user, and the context of the user. 

INPUT: {text}
Output should be a JSON object like this:

{
intent: XX,
response: XX
}

"""

### Error correction